# Furax Mapmaking Tutorial: First FURAX maps

This tutorial demonstrates how to use Furax's mapmaking facilities with two mapmakers. We'll cover:

1. Reading data from TOAST format
2. Setting up mapmaking configurations for MapMakers
3. Running BinnedMapMaker and MLMapMaker
4. Visualizing the results

## Prerequisites

Make sure you have installed Furax with mapmaking dependencies:
```bash
pip install -e '.[mapmaking]'
```
See the previous tutorial for setting up the environment for FURAX mapmaking.

In [ ]:
# Import necessary packages
from pathlib import Path

import jax
import matplotlib.pyplot as plt
import numpy as np

# TOAST import
import toast

# Furax imports
from furax.interfaces.toast.observation import ToastObservation
from furax.mapmaking import MapMakingConfig
from furax.mapmaking.config import LandscapeConfig, Landscapes, Methods, SolverConfig
from furax.mapmaking.mapmaker import MapMaker
from furax.mapmaking.pixell_utils import get_healpix_lonlat_ranges, plot_cartview

# Allow double-precision floats
# cf: https://docs.jax.dev/en/latest/notebooks/Common_Gotchas_in_JAX.html#double-64bit-precision
jax.config.update('jax_enable_x64', True)

# Set matplotlib style for better plots
plt.rcParams.update(
    {
        'font.family': 'serif',
        'mathtext.fontset': 'dejavuserif',
        'font.size': 11,
        'axes.labelsize': 11,
        'legend.fontsize': 10,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'font.sans-serif': 'Computer Modern Roman',
        'figure.dpi': 150,
        'figure.figsize': (4, 3),  # Put journal's text width here
        'savefig.bbox': 'tight',
    }
)

print(f'JAX devices: {jax.devices()}')
print('Furax mapmaking tutorial ready!')

## 1. Observation Data Interface

Let's load the test observation data that comes with the repository:

In [ ]:
# Load the test observation data from the repository using proper TOAST operators

# Check if the test data file exists
test_data_path = Path('./test_obs.h5')
if not test_data_path.exists():
    raise FileNotFoundError(
        f'Test data file {test_data_path} not found.'
        f"Make sure you're running from the repository root."
    )

print(f'Loading test observation data from {test_data_path}')
print(f'File size: {test_data_path.stat().st_size / 1024**2:.1f} MB')

# Load the TOAST data using proper TOAST operators
# Create empty TOAST data container
toast_data = toast.Data()

# Use TOAST's LoadHDF5 operator to load the data
# Use files parameter instead of volume for single file
loader = toast.ops.LoadHDF5(files=[str(test_data_path)])
loader.apply(toast_data)

print('\nTOAST data loaded successfully using LoadHDF5 operator!')
print(f'Number of observations: {len(toast_data.obs)}')

# Examine the first observation
obs = toast_data.obs[0]
print('\nObservation details:')
print(f'  Name: {obs.name}')

# Access focalplane info through 'telescope'
try:
    if hasattr(obs, 'telescope') and obs.telescope is not None:
        focalplane = obs.telescope.focalplane
        # Use detector_data.keys() instead of len(focalplane)
        detector_names = list(focalplane.detector_data['name'])
        print(f'  Detectors: {len(detector_names)} detectors')
        print(f'  Detector names: {detector_names[:5]}...')  # Show first 5
        print(f'  Focalplane sample rate: {focalplane.sample_rate}')
    else:
        print('  No telescope/focalplane information available in observation')
        print('  Will get detector info from Furax wrapper...')
except Exception as e:
    print(f'  Unexpected error accessing focalplane: {e}')
    print('  Will get detector info from Furax wrapper...')

# Check what data is available
print(f'  Shared data keys: {list(obs.shared.keys())}')
print(f'  Detector data keys: {list(obs.detdata.keys())}')

# Get basic properties
if 'times' in obs.shared:
    n_samples = len(obs.shared['times'].data)
    duration = (obs.shared['times'].data[-1] - obs.shared['times'].data[0]) / 3600  # hours
    print(f'  Number of samples: {n_samples}')
    print(f'  Observation duration: {duration:.2f} hours')

Now, lets load the observation using FURAX's observation interface. All functions in the GroundObservationData are compatible with other data structures including sotodlib.

In [ ]:
# Create FURAX observation wrapper
furax_obs = ToastObservation(
    data=toast_data,
    det_selection=None,  # Use all detectors
)

print('\nFurax observation wrapper created successfully')
print(f'  Number of detectors: {furax_obs.n_detectors}')
print(f'  Number of samples: {furax_obs.n_samples}')
print(f'  Sample rate: {furax_obs.sample_rate:.1f} Hz')

# Get TOD data for visualization
tod_data = furax_obs.get_tods()
times = furax_obs.get_elapsed_times()
det_names = furax_obs.detectors

print('\nTOD data extracted:')
print(f'  TOD shape: {tod_data.shape}')
print(f'  Data type: {tod_data.dtype}')
print(f'  RMS: {np.std(tod_data):.6f}')

# Show detector names
print(f'\nFirst few detector names: {det_names[:5]}')

Let's visualize the TOD data to understand what the mapmakers will process:

In [ ]:
# Visualize the real TOAST TOD data
print('=== FURAX Obs Info & Visualization ===\n')

tod_data = furax_obs.get_tods()
times = furax_obs.get_elapsed_times()
n_dets = furax_obs.n_detectors
n_samples = furax_obs.n_samples
sample_rate = furax_obs.sample_rate

# Analyze the real data characteristics
print('TOD Characteristics:')
print(f'  Number of detectors: {n_dets}')
print(f'  Samples per detector: {n_samples}')
print(f'  Sampling rate: {sample_rate:.1f} Hz')
print(f'  Observation duration: {times[-1] / 3600:.2f} hours')
print(f'  Total data points: {tod_data.size:,}')

# Show detector-wise statistics
print('\nDetector Statistics:')
for i in range(min(3, n_dets)):
    det_rms = np.std(tod_data[i, :])
    det_mean = np.mean(tod_data[i, :])
    print(f'  {det_names[i]}: mean={det_mean:.6f}, RMS={det_rms:.6f}')

if n_dets > 3:
    print(f'  ... and {n_dets - 3} more detectors')

# Estimate scan information
az = np.degrees(furax_obs.get_azimuth())
el = np.degrees(furax_obs.get_elevation())
print('\nScanning Information:')
print(f'  Azimuth range: [{np.min(az):.1f}, {np.max(az):.1f}] degrees')
print(f'  Elevation range: [{np.min(el):.1f}, {np.max(el):.1f}] degrees')

# Create visualization of the real TOD data
fig, ax = plt.subplots(1, 1, figsize=(15, 6))

# Plot: All detectors with offset (first 5 minutes)
time_mask = times < 300  # First 5 minutes
ax.set_title('All Detectors, First 5 Minutes (Offset for Clarity)')
n_show = min(8, n_dets)  # Show max 8 detectors
for i in range(n_show):
    offset = i * 3 * np.std(tod_data[i, :])  # Scale offset by RMS
    ax.plot(
        times[time_mask] / 60,
        tod_data[i, time_mask] + offset,
        alpha=0.7,
        linewidth=0.5,
        label=f'{det_names[i]}',
    )
ax.set_xlabel('Time [min]')
ax.set_ylabel('Signal (offset)')
ax.set_xlim([0, 5])
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Mapmaking Configuration

Different MapMakers are initialised through setting configuration parameters. Let's create configurations for Binned and Maximum-Likelihood (ML) mapmakers in this tutorial:

- **BinnedMapMaker**: Uses diagonal noise covariance (white noise), requires `binned=True`
- **MLMapMaker**: Uses full noise model (atmospheric noise), requires `binned=False`

In [ ]:
# Configuration for BinnedMapMaker
# Only includes fields relevant to BinnedMapMaker
binned_config = MapMakingConfig(
    # Required: Method and binned flag
    method=Methods.BINNED,
    binned=True,  # MUST be True for BinnedMapMaker
    # Map pixelization
    landscape=LandscapeConfig(
        type=Landscapes.HPIX,  # Use HEALPix pixelization
        nside=64,  # HEALPix nside parameter (resolution)
    ),
    # Currently, the mapmakers don't work for demodulated data
    demodulated=False,  # Whether data is already demodulated
    double_precision=True,  # Use single precision for faster computation
    # Sample selection critera ('bad' samples are discarded)
    scanning_mask=True,  # Apply scanning mask (discard samples at turnarounds)
    sample_mask=False,  # Apply sample mask (additionally, discard flagged samples)
    # Pointing computation
    pointing_on_the_fly=True,  # Memory efficient pointing (recommended)
    # Noise model (BinnedMapMaker uses white noise)
    fit_noise_model=True,  # Fit noise model from data
    # Optional: Debug mode. Output includes additional products for debugging
    debug=True,
)

print('BinnedMapMaker configuration:')
print(f'  Method: {binned_config.method}')
print(f'  Binned: {binned_config.binned} (required for BinnedMapMaker)')
print(f'  Landscape: {binned_config.landscape.type}, nside={binned_config.landscape.nside}')
print(f'  Demodulated: {binned_config.demodulated}')
print(f'  Scanning mask: {binned_config.scanning_mask}')

In [ ]:
# Configuration for MLMapMaker
# Only includes fields relevant to MLMapMaker
ml_config = MapMakingConfig(
    # Required: Method and ML flag
    method=Methods.MAXL,  # Maximum Likelihood method
    binned=False,  # MUST be False for MLMapMaker
    # Map pixelization
    landscape=LandscapeConfig(
        type=Landscapes.HPIX,  # Use HEALPix pixelization
        nside=64,  # HEALPix nside parameter (resolution)
    ),
    # Currently, the mapmakers don't work for demodulated data
    demodulated=False,  # Whether data is already demodulated
    double_precision=True,  # Use double precision for ML
    # Sample selection critera ('bad' samples are discarded)
    scanning_mask=True,  # Apply scanning mask (discard samples at turnarounds)
    sample_mask=False,  # Apply sample mask (additionally, discard flagged samples)
    # Map pixel selection criteria ('bad' pixels are discarded)
    hits_cut=1e-3,  # Minimum hits per pixel, as a fraction of the maximum hits value
    cond_cut=1e-3,  # Minimum condition number
    # Pointing computation
    pointing_on_the_fly=True,  # Memory efficient pointing (recommended)
    # Noise model parameters (MLMapMaker uses atmospheric noise model)
    fit_noise_model=True,  # Fit noise model from data
    correlation_length=1000,  # Noise correlation length (samples)
    nperseg=1024,  # FFT segment length for PSD estimation
    psd_fmin=1e-2,  # Minimum frequency for PSD fitting
    # Solver configuration (ML requires iterative solving (PCG))
    solver=SolverConfig(
        rtol=1e-6,  # Relative tolerance
        atol=0,  # Absolute tolerance
        max_steps=1000,  # Maximum iterations
    ),
    # Optional: Debug mode. Output includes additional products for debugging
    debug=True,
)

print('MLMapMaker configuration:')
print(f'  Method: {ml_config.method}')
print(f'  Binned: {ml_config.binned} (must be False for MLMapMaker)')
print(f'  Landscape: {ml_config.landscape.type}, nside={ml_config.landscape.nside}')
print(f'  Demodulated: {ml_config.demodulated}')
print(f'  Noise correlation length: {ml_config.correlation_length}')
print(f'  Solver tolerance: rtol={ml_config.solver.rtol}, atol={ml_config.solver.atol}')

## 4. Mapmaking Runs

Let's create MapMaker instances using our mapmaking configurations and apply them on the data.

- Use `MapMaker.from_config()` to create a MapMaker instance from a given configuration.

- Use `make_maps()` to run a mapmaker on a given observation, optionally saving output to files.

In [ ]:
print('=== MapMaker Execution with TOAST Data ===\n')

# Provide output paths to save the results
binned_out_dir = None
ml_out_dir = None

# Test BinnedMapMaker configuration and creation
print('1. BinnedMapMaker:')
try:
    binned_mapmaker = MapMaker.from_config(binned_config)
    print(f'   ✓ {binned_mapmaker.__class__.__name__} created successfully')
    print(f'   ✓ Configuration validated: binned={binned_config.binned}')

    # Actually run the mapmaker on data
    print('   → Running mapmaking with TOAST data...')
    binned_results = binned_mapmaker.run(furax_obs, out_dir=binned_out_dir)
    print('   ✓ Mapmaking completed successfully!')
    print(f'   ✓ Output maps shape: {binned_results["map"].shape}')
    print(f'   ✓ Weights shape: {binned_results["weights"].shape}')
    binned_success = True

except Exception as e:
    print(f'   ✗ Error with BinnedMapMaker: {e}')
    binned_success = False
    binned_results = None

print()

# Test MLMapMaker configuration and creation
print('2. MLMapMaker:')
try:
    ml_mapmaker = MapMaker.from_config(ml_config)
    print(f'   ✓ {ml_mapmaker.__class__.__name__} created successfully')
    print(f'   ✓ Configuration validated: binned={ml_config.binned}')

    # Actually run the mapmaker on real data
    print('   → Running mapmaking with TOAST data...')
    ml_results = ml_mapmaker.run(furax_obs, out_dir=ml_out_dir)
    print('   ✓ Mapmaking completed successfully!')
    print(f'   ✓ Output maps shape: {ml_results["map"].shape}')
    print(f'   ✓ Weights shape: {ml_results["weights"].shape}')
    ml_success = True

except Exception as e:
    print(f'   ✗ Error with MLMapMaker: {e}')
    ml_success = False
    ml_results = None

print()
print('=== Mapmaking Results Summary ===')

if binned_success:
    print('✓ BinnedMapMaker: Successfully created maps from TOAST data')

if ml_success:
    print('✓ MLMapMaker: Successfully created maps real TOAST data')


print('\n' + '=' * 60)
print('=' * 60)

## 5. Output Visualisation

Let's see what the output would look like from both mapmakers.

In [ ]:
# Output is a dictionary
print('Binned Mapmaker output products:', list(binned_results.keys()))
print('ML Mapmaker output products:', list(ml_results.keys()))

# Note you can load all results from a mapmaking run directory
if False:
    from furax.interfaces.sotodlib.mapmaker import load_result

    results = load_result('/path/to/mapmaking/results/')

- `map`: the output sky map. Has shape (n_stokes, n_pixels)

- `weights`: the output sky map weights. Has shape (n_pixels, n_stokes, n_stokes)

- `noise_fit`: the fitted noise model, available when fit_noise_model=True. Has shape (n_detectors, n_noise_fit_parameters)

- `proj_...`: the TOD projection of the fitted data, available when debug=True. For ML, `projs` is a dictionary containing projections of various projecctions.

- `weights_uncut`: for ML mapmaker only, contains the sky map weights BEFORE the pixel selection using hits_cut and cond_cut.

In [ ]:
# Output maps are usually non-zero on a localised region of the sky
# We use the II component ([...,0,0]) of the weights to find this region
lonra, latra = get_healpix_lonlat_ranges(binned_results['weights'][:, 0, 0], padding_deg=3)

# Plot the maps using CAR projection
fig, axs = plot_cartview(
    binned_results['map'],
    titles=['Binned I', 'Binned Q', 'Binned U'],
    lonra=lonra,
    latra=latra,
    cmap='RdBu',
    nside=64,
)

fig, axs = plot_cartview(
    ml_results['map'],
    titles=['ML I', 'ML Q', 'ML U'],
    lonra=lonra,
    latra=latra,
    cmap='RdBu',
    nside=64,
)

## Summary

This tutorial covered the key aspects of using Furax's mapmaking facilities:

### Key Takeaways

1. **Observation Data Interface**:
   - Use `ToastObservation` or `SotodlibObservation` to wrap the observational data to be used for mapmaking

2. **Mapmaking Configuration**:
   - Create a `MapMakingConfig` instance with appropriate parameters suitable for the mapmaker to be used
   - Make sure to specify relevant settings: sky landscape, sample selection, pointing computation, noise model, pixel selection, iterative solver settings, ...

3. **Mapmaking Runs**
   - Initialise a mapmaker instance using `MapMaker.from_config()`
   - Run the mapmaker using `mapmaker.make_maps(furax_obs, out_dir)`

4. **Output Visualisation**:
   - Load the outputs from a directory using `load_results` if needed
   - The `map` and `weights` keys contain the main results of mapmaking 
   - Some plotting utilities are also provided

### Next Steps

- To be continued in the following notebooks (being written...):
   - Run on various mapmaking settings: sotodlib (data interface), astropy's WCS (landscape), adding templates (mapmaker), etc
   - Use the command-line interface to run mapmaking
   - Learn how to submit batch jobs on NERSC and Jean-Zay

---

**🎉 Tutorial Complete!** You now know how to configure and use Furax mapmakers properly.